# Superior Computer Chess with Model Predictive Control, Reinforcement Learning, and Rollout

This is a sample code for the MPC-MC method with one-step lookahead introduced in the paper. MPC/MC based on Stockfish and/or Komodo Dragon plays against Stockfish and/or Komodo Dragon.

The codes have been tested on Google Colab with High-RAM TPU.

Code copyright @ Yuchao Li

Please cite the paper if the codes are used for academic publication

Variations of implementaion can be obtained upon request by writing to
yuchaoli@asu.edu

# Setup of the Environment

This section downloads and installs chess engines Stockfish and freely available version of Komodo Dragon, as well as other packages

In [ ]:
!pip install python-chess

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 1.6 MB/s eta 0:00:00


In [ ]:
!pip install livelossplot==0.3.4

  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
Using cached jedi-0.19.1-py2.py3-none-any.whl (1.6 MB)


In [ ]:
!wget https://github.com/official-stockfish/Stockfish/releases/latest/download/stockfish-ubuntu-x86-64-sse41-popcnt.tar

--2024-09-10 15:52:06--  https://github.com/official-stockfish/Stockfish/releases/latest/download/stockfish-ubuntu-x86-64-sse41-popcnt.tar
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/official-stockfish/Stockfish/releases/download/sf_17/stockfish-ubuntu-x86-64-sse41-popcnt.tar [following]
--2024-09-10 15:52:07--  https://github.com/official-stockfish/Stockfish/releases/download/sf_17/stockfish-ubuntu-x86-64-sse41-popcnt.tar
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://objects.githubusercontent.com/github-production-release-asset-2e65be/20976138/9a27337c-3507-4f41-84f4-92199c4cb1d9?X-Amz-Algorithm=AWS4-HMAC-SHA256&X-Amz-Credential=releaseassetproduction%2F20240910%2Fus-east-1%2Fs3%2Faws4_request&X-Amz-Date=20240910T155207Z&X-Amz-Expires=300&X-Amz-Signature=eb8cbed4da

In [ ]:
!tar -xf stockfish-ubuntu-x86-64-sse41-popcnt.tar

In [ ]:
!chmod +x stockfish/stockfish-ubuntu-x86-64-sse41-popcnt

In [ ]:
!wget https://komodochess.com/pub/dragon.zip

--2024-09-10 15:52:07--  https://komodochess.com/pub/dragon.zip
Resolving komodochess.com (komodochess.com)... 165.22.25.249
Connecting to komodochess.com (komodochess.com)|165.22.25.249|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 85049133 (81M) [application/zip]
Saving to: ‘dragon.zip’

dragon.zip          100%[===================>]  81.11M  4.00MB/s    in 36s     

2024-09-10 15:52:44 (2.25 MB/s) - ‘dragon.zip’ saved [85049133/85049133]



In [ ]:
!unzip dragon.zip -d komodo_chess

Archive:  dragon.zip
   creating: komodo_chess/dragon_05e2a7/
 extracting: komodo_chess/dragon_05e2a7/Syzygy7.pdf  
   creating: komodo_chess/dragon_05e2a7/Linux/
 extracting: komodo_chess/dragon_05e2a7/Linux/dragon-linux-avx2  
 extracting: komodo_chess/dragon_05e2a7/Linux/dragon-linux  
 extracting: komodo_chess/dragon_05e2a7/READMEDragon.html  
   creating: komodo_chess/dragon_05e2a7/Windows/
 extracting: komodo_chess/dragon_05e2a7/Windows/dragon-64bit.exe  
 extracting: komodo_chess/dragon_05e2a7/Windows/dragon-64bit-avx2.exe  
   creating: komodo_chess/dragon_05e2a7/OSX/
 extracting: komodo_chess/dragon_05e2a7/OSX/dragon-avx2-osx  
 extracting: komodo_chess/dragon_05e2a7/OSX/dragon-osx  
 extracting: komodo_chess/dragon_05e2a7/setHash.txt  


In [ ]:
!chmod +x komodo_chess/dragon_05e2a7

In [ ]:
!pip install cairosvg

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 708.2 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 1.4 MB/s eta 0:00:00


In [ ]:
# Optional: Mount personal Google drive for recording test results
# from google.colab import drive
# drive.mount('/content/drive')

# Initialization

Import dependencies and set parameter values. The parameters include the engines used and played against, the strength of the engines.

In [ ]:
import os
import chess
import chess.svg
import chess.engine
import chess.pgn
import concurrent.futures
import cairosvg
import gc
import time
from collections import defaultdict
import io
import sys

In [ ]:
sk_engine_path = "stockfish/stockfish-ubuntu-x86-64-sse41-popcnt"
ko_engine_path = "komodo_chess/dragon_05e2a7/Linux/dragon-linux"
engines_path = {'SK':sk_engine_path, 'Ko':ko_engine_path}


# Set the skill level (0-20 where 0 is the weakest and 20 is the strongest)
ev_level = 20           # evaluation engine level
op_level = 20           # nominal opponent engine level
exp_level = 20          # expert engine level, used in fortified version of rollout
levels = (ev_level,op_level,exp_level)

actual_op_level = 20    # actual opponent level

# Specify the type of engines used. 'SK' stands for Stockfish, 'Ko' stands for Komodo Dragon
ev_type = 'SK'
op_type = 'SK'
exp_type = 'SK'
actual_op_type = 'SK'

engines_config = [(ev_type, ev_level), (op_type, op_level), (exp_type, exp_level)]
actual_op_config = (actual_op_type, actual_op_level)

limit_ev = chess.engine.Limit(
    time=0.5        # Maximum time in seconds for evaluation engine
)
limit_op = chess.engine.Limit(
    time=0.5       # Maximum time in seconds for nominal opponent engine
)
limit_exp = chess.engine.Limit(
    time=0.5         # Maximum time in seconds for expert engine
)
limits = (limit_ev,limit_op,limit_exp)
limit_actual_op = chess.engine.Limit(
    time=0.5         # Maximum time in seconds for actual opponent engine
)
engines_list_len = 0 # Used to test the effect of hashtable. Set to 0 to remove hashtables.


# Documenting Functions

This section define the functions used to save the board images and the .pgn file during the game.

In [ ]:
def save_board_image(board, frame_count, output_dir='screenshots'):
    # Generate the SVG image of the board
    svg_data = chess.svg.board(board=board)
    # Define the image path
    image_path = os.path.join(output_dir, f'frame_{frame_count:04d}.png')
    # Convert SVG to PNG and save
    cairosvg.svg2png(bytestring=svg_data, write_to=image_path)

In [ ]:
def generate_pgn_from_fen(history_fen, output_dir):
    if not history_fen:
        print("The history_fen list is empty.")
        return

    # Create an initial board
    board = chess.Board(history_fen[0])

    # Create a game
    game = chess.pgn.Game()

    # Start setting up the board from the initial FEN
    node = game

    for fen in history_fen[1:]:
        next_board = chess.Board(fen)
        move = None

        # Find the move that leads to the next_board
        for candidate_move in board.legal_moves:
            board.push(candidate_move)
            if board.fen() == fen:
                move = candidate_move
                board.pop()
                break
            board.pop()

        if move is None:
            print("Unable to find move for FEN:", fen)
            return

        # Add the move to the game
        node = node.add_variation(move)
        board.push(move)

    # Save the game to a PGN file
    with open(output_dir, 'w') as pgn_file:
        exporter = chess.pgn.FileExporter(pgn_file)
        game.accept(exporter)


# Utility Functions

This section defines various utility functions used in MPC-MC, or as the opponent chess engine.

In [ ]:
def create_engine(engine_path):
    engine = chess.engine.SimpleEngine.popen_uci(engine_path)
    return engine

def game_complete(board):
    if board.is_fifty_moves():
        return True
    elif board.is_repetition(3):
        return True
    else:
        return board.is_game_over(claim_draw=False)

def game_complete_reason(board):
    if board.is_checkmate():
        print("Game ended by checkmate.")
    elif board.is_stalemate():
        print("Game ended by stalemate.")
    elif board.is_insufficient_material():
        print("Game ended by insufficient material.")
    elif board.is_seventyfive_moves():
        print("Game ended by seventy-five moves rule.")
    elif board.is_fivefold_repetition():
        print("Game ended by fivefold repetition.")
    elif board.is_repetition(3):
        print("Game is claimed as draw by threefold repetition.")
    elif board.is_fifty_moves():
        print("Game is claimed as draw by fifty-move rule.")

In [ ]:
def best_move(engine, board, limit = chess.engine.Limit(time=2.0)):
    # Get the best move
    # result = engine.play(board, limit=limit)
    # best_move = result.move
    evaluation = engine.analyse(board, limit, multipv=1)
    best_move = evaluation[0]['pv'][0]
    return best_move

In [ ]:
def top_moves(engine, board, limit = chess.engine.Limit(time=2.0), top_num = 3):
     evaluation = engine.analyse(board, limit, multipv=top_num)
     top_moves = [move['pv'][0] for move in evaluation]
     return top_moves

In [ ]:
def evaluation_score(engine, board, limit = chess.engine.Limit(time=2.0)):
    info = engine.analyse(board, limit=limit)
    if game_complete(board):
        game_result = board.result()
        if game_result == '1-0':
            return float('inf')  # Checkmate for white
        elif game_result == '0-1':
            return float('-inf')  # Checkmate for black
        else:
            return  0.0  # Draw
    else:
        score = info['score'].white()  # Relative to white
        if score.is_mate():
            return float(999999 / score.mate()) if score.mate() > 0 else -float(999999 / abs(score.mate()))
        else:
            return score.score()

In [ ]:
def sim_one_round_base(engine, board, limit = chess.engine.Limit(time=2.0)):

    sim_board = board.copy()

    sk_move = best_move(engine, sim_board, limit=limit)  # Get the engine's response
    if sk_move:  # Make sure the engine has a valid response
        sim_board.push(sk_move)
    else:
        # Handle the case where the engine does not provide a valid response
        print("Warning: Response move is None.")
        return sim_board, None

    # Directly evaluate the board using the engine
    try:
        sim_score = evaluation_score(engine, sim_board, limit=limit)
    except Exception as e:
        print(f"Error evaluating board: {e}")
        sim_score = None

    # Check if the evaluation score is None
    if sim_score is None:
        print("Warning: Evaluation score is None.")

    if game_complete(sim_board):
        resp_move = None
    else:
        resp_move = best_move(engine, sim_board, limit=limit)
    return sim_board, sim_score, sk_move, resp_move

In [ ]:
def engine_close(engine, flag):
    if flag:
        engine.quit()

In [ ]:
def single_engine_generator(engines_path,engine_config):
    engine_type, engine_level = engine_config
    engine_path = engines_path[engine_type]
    engine = create_engine(engine_path)
    if engine_type == 'SK':
        engine.configure({"Skill Level": engine_level})
    return engine

In [ ]:
def engines_generator(engines_path, engines_config, engines):
    # Generate engines = (engine_ev, engine_op, engine_exp)
    if engines is None:
        engines = ()
        for i in range(len(engines_config)):
            engine = single_engine_generator(engines_path, engines_config[i])
            engines += (engine,)
    return engines

In [ ]:
def engines_list_generator(engines_path, engines_config, engines_list_len):
    engines_list = []
    for _ in range(engines_list_len):
        engines = engines_generator(engines_path, engines_config, None)
        engines_list.append(engines)
    return engines_list

In [ ]:
def engines_close(engines,flag):
    if flag:
        closed_engines = set()
        for engine in engines:
            if engine not in closed_engines:
                closed_engines.add(engine)
                engine.close()

In [ ]:
def engines_list_close(engines_list):
    for engines in engines_list:
        engines_close(engines, True)

In [ ]:
def parallel_best_response(engines_path, engine_config, board, limit = chess.engine.Limit(time=2.0)):
    # This function is used so that the computing resources assigned to the actual opponent is close to
    # that assigned to evaluate one legal move in MPC-MC
    legal_moves = list(board.legal_moves)

    def simulate_response(params):
        engines_path, engine_config, board, move, limit = params.values()
        engine = single_engine_generator(engines_path, engine_config)
        op_move = best_move(engine, board, limit = limit)
        engine.close()
        return op_move

    # Create a list of tuples with parameters for each move
    params_list = [{'engines_path': engines_path, 'engine_config': engine_config, 'board': board.copy(), 'move': move, 'limit': limit} for move in legal_moves]

    # Run the simulations in parallel
    with concurrent.futures.ThreadPoolExecutor() as executor:
        results = list(executor.map(simulate_response, params_list))

    return results[0]

# One Step Rollout
This section defines the MPC-MC with one-step lookahead and its fortified variant.

In [ ]:
def sim_one_round(engines_path, engines_config, board, move, limits, engines = None, verbose = False):
    ev_limit, op_limit, exp_limit = limits
    flag = engines is None
    engines = engines_generator(engines_path, engines_config, engines)
    engine_ev, engine_op, engine_exp = engines
    start_time = time.perf_counter()
    sim_board = board.copy()
    if move == 'SK':
        sim_board, sim_score, sk_move, resp_move = sim_one_round_base(engine_exp, sim_board, limit=exp_limit)
        sim_move = sk_move
        end_time = time.perf_counter()
        info = f"SK Move {sk_move} Score {sim_score} evaluated in {end_time - start_time:0.4f} seconds"
        if verbose:
            print(info)
        engines_close(engines, flag)
        return sim_board, sim_score, sim_move, resp_move
    elif move == 'EV':
        sim_board, sim_score, sk_move, resp_move = sim_one_round_base(engine_exp, sim_board, limit=exp_limit)
        sim_move = sk_move
        end_time = time.perf_counter()
        info = f"SK Move {sk_move} Score {sim_score} evaluated in {end_time - start_time:0.4f} seconds"
        if verbose:
            print(info)
        engines_close(engines, flag)
        return sim_board, 'EV', sim_move, resp_move

    sim_move = move
    sim_board.push(move)  # Play the initial move

    # Check if the game is over after the initial move
    if game_complete(sim_board):
        sim_score = evaluation_score(engine_ev, sim_board, limit=ev_limit)
        engines_close(engines, flag)
        return sim_board, sim_score, sim_move, None

    resp_move = best_move(engine_op, sim_board, limit=op_limit)  # Get the engine's response
    if resp_move:  # Make sure the engine has a valid response
        sim_board.push(resp_move)
    else:
        # Handle the case where the engine does not provide a valid response
        print("Warning: Response move is None.")
        engines_close(engines, flag)
        return sim_board, None, sim_move, None
    # Directly evaluate the board using the engine
    try:
        sim_score = evaluation_score(engine_ev, sim_board, limit=ev_limit)
    except Exception as e:
        print(f"Error evaluating board: {e}")
        sim_score = None

    # Check if the evaluation score is None
    if sim_score is None:
        print(f"Move is {move} and Score is {sim_score}!")
        print("Warning: Evaluation score is None.")
    engines_close(engines, flag)
    end_time = time.perf_counter()
    info = f"Move {move} Score {sim_score} evaluated in {end_time - start_time:0.4f} seconds Response Move is {resp_move}"
    if verbose:
        print(info)

    return sim_board, sim_score, sim_move, resp_move

In [ ]:
def parallel_rollout_one_step_fix_opponent(engines_path, engines_config, board, limits, engines_list = [], verbose=False, white = True):
    # Parallel implementation of MPC-MC with one-step lookahead

    legal_moves = list(board.legal_moves)

    def simulate_move(params):
        engines_path, engines_config, board_copy, move, limits, engines, verbose = params.values()
        sim_board, sim_score, sim_move, resp_move = sim_one_round(engines_path, engines_config, board_copy, move, limits, engines = engines, verbose = verbose)
        return sim_move, sim_score, resp_move

    # Create a list of tuples with parameters for each move
    params_list = [{'engines_path': engines_path, 'engines_config': engines_config, 'board': board.copy(), 'move': move, 'limits': limits, 'engines': None, 'verbose': verbose} for move in legal_moves]
    params_list.append({'engines_path': engines_path, 'engines_config': engines_config, 'board': board.copy(), 'move': 'EV', 'limits': limits, 'engines': None, 'verbose': verbose})

    for i in range(min([len(params_list),len(engines_list)])):
        params_list[i]['engines'] = engines_list[i]

    # Run the simulations in parallel
    with concurrent.futures.ThreadPoolExecutor() as executor:
        results = list(executor.map(simulate_move, params_list))

    # Find the pair (sim_move, False), assign sim_move to sk_move, and remove the pair from results
    for result in results:
        if result[1] == 'EV':
            sk_move = result[0]
            results.remove(result)
            break

    # Evaluate the results
    if white:
        rollout_move, rollout_score, resp_move = max(results, key=lambda x: (x[1], isinstance(x[1], float)))
    else:
        rollout_move, rollout_score, resp_move = min(results, key=lambda x: (x[1], isinstance(x[1], float)))

    if verbose:
        print('-' * 20)
        print(f"SK Move: {sk_move}")
        print(f"Rollout Move: {rollout_move}, Rollout Score: {rollout_score}")
        print('-' * 20)

    return rollout_move, rollout_score, resp_move

In [ ]:
def parallel_fortified_rollout_one_step_fix_opponent(engines_path, engines_config, board, limits, engines_list = [], verbose=False, white = True):
    # Parallel implementation of fortified variant of MPC-MC with one-step lookahead

    legal_moves = list(board.legal_moves)

    def simulate_move(params):
        engines_path, engines_config, board_copy, move, limits, engines, verbose = params.values()
        sim_board, sim_score, sim_move, resp_move = sim_one_round(engines_path, engines_config, board_copy, move, limits, engines = engines, verbose = verbose)
        return sim_move, sim_score, resp_move

    # Create a list of tuples with parameters for each move
    params_list = [{'engines_path': engines_path, 'engines_config': engines_config, 'board': board.copy(), 'move': move, 'limits': limits, 'engines': None, 'verbose': verbose} for move in legal_moves]
    params_list.append({'engines_path': engines_path, 'engines_config': engines_config, 'board': board.copy(), 'move': 'EV', 'limits': limits, 'engines': None, 'verbose': verbose})
    params_list.append({'engines_path': engines_path, 'engines_config': engines_config, 'board': board.copy(), 'move': 'SK', 'limits': limits, 'engines': None, 'verbose': verbose})

    for i in range(min([len(params_list),len(engines_list)])):
        params_list[i]['engines'] = engines_list[i]

    # Run the simulations in parallel
    with concurrent.futures.ThreadPoolExecutor() as executor:
        results = list(executor.map(simulate_move, params_list))

    # Find the pair (sim_move, False), assign sim_move to sk_move, and remove the pair from results
    for result in results:
        if result[1] == 'EV':
            sk_move = result[0]
            results.remove(result)
            break

    # Evaluate the results
    if white:
        rollout_move, rollout_score, resp_move = max(results, key=lambda x: (x[1], isinstance(x[1], float)))
    else:
        rollout_move, rollout_score, resp_move = min(results, key=lambda x: (x[1], isinstance(x[1], float)))

    if verbose:
        print('-' * 20)
        print(f"SK Move: {sk_move}")
        print(f"Rollout Move: {rollout_move}, Rollout Score: {rollout_score}")
        print('-' * 20)

    return rollout_move, rollout_score, resp_move

# Rollout Game Play

The following function is used for applying MPC-MC plays against an chess engine.

In [ ]:
def rollout_chess_play(rollout, engines_path, engines_config, limits, engines_list_len = 0, actual_op_config = None, limit_actual_op = chess.engine.Limit(time=2.0), deterministic = True, verbose=False, white=True, record = True, output_dir='output.pgn', log_dir = 'output.txt'):
    engines_list = engines_list_generator(engines_path, engines_config, engines_list_len)
    if actual_op_config is None:
        _,actual_op_config,_ = engines_config

    board = chess.Board()
    history_fen = [board.fen()]
    round = 1
    # Create a StringIO object to capture the prints
    if record:
        output = io.StringIO()
        old_stdout = sys.stdout
        sys.stdout = output


    resp_move = parallel_best_response(engines_path, actual_op_config, board, limit = limit_actual_op)

    while not game_complete(board):
        if verbose:
            print('*'*20)
            print(f"Round {round}:")
        round += 1

        if white:
            rollout_move, rollout_score, resp_move = rollout(engines_path, engines_config, board, limits, engines_list = engines_list, verbose=verbose, white=white)
            board.push(rollout_move)
            # Capture board state after rollout move
            history_fen.append(board.fen())
            if game_complete(board):
                break
            op_move = parallel_best_response(engines_path, actual_op_config, board, limit = limit_actual_op)
            if deterministic:
                op_move = resp_move

            board.push(op_move)
            # Capture board state after engine move
            history_fen.append(board.fen())
        else:
            op_move = parallel_best_response(engines_path, actual_op_config, board, limit = limit_actual_op)
            if deterministic:
                op_move = resp_move
            board.push(op_move)
            # Capture board state after engine move
            history_fen.append(board.fen())
            if game_complete(board):
                #engine_actual_op.close()
                break
            rollout_move, rollout_score, resp_move = rollout(engines_path, engines_config, board, limits, engines_list = engines_list, verbose=verbose, white=white)
            board.push(rollout_move)
            # Capture board state after rollout move
            history_fen.append(board.fen())

    engines_list_close(engines_list)

    if game_complete(board):
        game_complete_reason(board)

    if record:
        sys.stdout = old_stdout
        output_string = output.getvalue()
        output.close()
        with open(log_dir, 'w') as file:
            file.write(output_string)

    generate_pgn_from_fen(history_fen, output_dir = output_dir)

# Game Tournament

This section provides an example of using the MPC-MC against a chess engine. The default parameters are for a 10-game tourament, where stochastic MPC-MC plays against an opponent engine whose move may not be predicted precisely by the nominal opponent in MPC-MC.

In [ ]:
location = '/content/' # location to store the game .pgn file and .txt file that records the printouts
deterministic = False # True for determinitic MPC-MC and False for stochastic MPC-MC

In [ ]:
game_num = 10
for i in range(0,game_num):
    idx = i+1
    if idx % 2 == 0:
        white=False # MPC-MC plays black
        player = '_b'
    else:
        white=True # MPC-MC plays white
        player = '_w'
    test_dir = location+'0d5step_L'+str(ev_level)+'_opL'+str(op_level)+'_'+str(idx)+player+'.pgn'
    log_dir = location+'0d5step_L'+str(ev_level)+'_opL'+str(op_level)+'_'+str(idx)+player+'.txt'
    rollout_chess_play(parallel_rollout_one_step_fix_opponent, engines_path, engines_config, limits, actual_op_config = actual_op_config, limit_actual_op =limit_actual_op, deterministic = deterministic, verbose=True, white=white, output_dir=test_dir, log_dir =log_dir)